In [ ]:
import json

# Load data from the JSON file
with open('Solar.json', 'r') as f:
    data = json.load(f)

# Extract main bounding box
main_box_data = data['boundingBox']
main_box = [
    main_box_data['sw']['latitude'],
    main_box_data['sw']['longitude'],
    main_box_data['ne']['latitude'],
    main_box_data['ne']['longitude']
]

# Extract solar panel data
solarPotential_data = data['solarPotential']
solar_panels_data = solarPotential_data.get('solarPanels', [])

# Print the number of solar panels
#print(f"Number of solar panels: {len(solar_panels_data)}")

# Extract roof segment statistics
roofSegmentStats_data = solarPotential_data.get('roofSegmentStats', [])

bounding_box_data = data['boundingBox']

sw_data = bounding_box_data['sw']
ne_data = bounding_box_data['ne']
sw_structure = {key: type(value).__name__ for key, value in sw_data.items()}
ne_structure = {key: type(value).__name__ for key, value in ne_data.items()}

sw_data = bounding_box_data['sw']
ne_data = bounding_box_data['ne']
sw_lat, sw_long = sw_data['latitude'], sw_data['longitude']
ne_lat, ne_long = ne_data['latitude'], ne_data['longitude']

# Extract heights of each sample and store in a list
sample_heights = []
for i, segment in enumerate(roofSegmentStats_data):
    height = segment.get('planeHeightAtCenterMeters', None)
    if height is not None:
        sample_heights.append(height)

# Extract sample boxes
sample_boxes = []
for sample in roofSegmentStats_data:
    box = sample.get('boundingBox', {})
    sw = box.get('sw', {})
    ne = box.get('ne', {})
    sample_box = [
        sw.get('latitude', None),
        sw.get('longitude', None),
        ne.get('latitude', None),
        ne.get('longitude', None)
    ]
    sample_boxes.append(sample_box)

# Dimensions of a single solar panel in inches
panel_width = 100
panel_height = 100

In [ ]:
import folium
import math

# Function to rotate a point
def rotate_point(x, y, angle_degrees, cx, cy):
    angle_radians = math.radians(angle_degrees)
    dx = x - cx
    dy = y - cy
    rotated_dx = dx * math.cos(angle_radians) - dy * math.sin(angle_radians)
    rotated_dy = dx * math.sin(angle_radians) + dy * math.cos(angle_radians)
    return cx + rotated_dx, cy + rotated_dy

# Calculate the midpoint of the bounding box
mid_lat, mid_long = (sw_lat + ne_lat) / 2, (sw_long + ne_long) / 2

# Calculate scaling factor based on latitude
scale_factor = math.cos(math.radians(mid_lat))

# Adjust the deltas based on the scaling factor
scaled_delta_lat = (ne_lat - sw_lat) / scale_factor
scaled_delta_long = ne_long - sw_long

# Check if rotation is needed
rotation_needed = abs(scaled_delta_lat - scaled_delta_long) > 0.01  # Set a suitable threshold

# Initialize points to original coordinates
rotated_sw = (sw_lat, sw_long)
rotated_ne = (ne_lat, ne_long)
rotated_se = (ne_lat, sw_long)
rotated_nw = (sw_lat, ne_long)

if rotation_needed:
    # Calculate the angle of rotation based on the adjusted deltas
    angle_degrees = math.degrees(math.atan2(scaled_delta_lat, scaled_delta_long))

    # Rotate each corner of the bounding box
    rotated_sw = rotate_point(sw_lat / scale_factor, sw_long, angle_degrees, mid_lat / scale_factor, mid_long)
    rotated_ne = rotate_point(ne_lat / scale_factor, ne_long, angle_degrees, mid_lat / scale_factor, mid_long)
    rotated_se = rotate_point(ne_lat / scale_factor, sw_long, angle_degrees, mid_lat / scale_factor, mid_long)
    rotated_nw = rotate_point(sw_lat / scale_factor, ne_long, angle_degrees, mid_lat / scale_factor, mid_long)

    # Rescale the rotated coordinates back to the original scale
    rotated_sw = (rotated_sw[0] * scale_factor, rotated_sw[1])
    rotated_ne = (rotated_ne[0] * scale_factor, rotated_ne[1])
    rotated_se = (rotated_se[0] * scale_factor, rotated_se[1])
    rotated_nw = (rotated_nw[0] * scale_factor, rotated_nw[1])

# Initialize the folium map centered around the bounding box
zoom_level = 6  # Adjust as needed
m = folium.Map(location=[mid_lat, mid_long], zoom_start=zoom_level)

# Draw the rotated bounding box as a polygon
folium.Polygon([rotated_sw, rotated_se, rotated_ne, rotated_nw, rotated_sw], 
               color='red').add_to(m)

# Show the map
m


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Function to calculate the Haversine distance between two points on the Earth's surface
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 3958.8  # Radius of Earth in miles
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2) * sin(dlon/2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c  # Distance in miles
    return distance * 63360  # Convert to inches

# Function to calculate the number of panels that can fit inside a bounding box
def panels_in_bounding_box(bounding_box, panel_width, panel_height):
    sw_lat, sw_lon, ne_lat, ne_lon = bounding_box

    # Calculate the width and height of the bounding box in inches using Haversine formula
    width = haversine_distance(sw_lat, sw_lon, sw_lat, ne_lon)
    height = haversine_distance(sw_lat, sw_lon, ne_lat, sw_lon)

    # Calculate the number of panels that can fit
    panels_in_width = int(width // panel_width)
    panels_in_height = int(height // panel_height)
    total_panels = panels_in_width * panels_in_height

    return total_panels

# Function to calculate the panel centers
def calculate_panel_centers(bounding_box, panel_width, panel_height):
    sw_lat, sw_lon, ne_lat, ne_lon = bounding_box

    # Calculate the number of panels in width and height
    panels_in_width = int(haversine_distance(sw_lat, sw_lon, sw_lat, ne_lon) // panel_width)
    panels_in_height = int(haversine_distance(sw_lat, sw_lon, ne_lat, sw_lon) // panel_height)

    # Calculate the step size for panel centers
    lat_step = (ne_lat - sw_lat) / panels_in_height
    lon_step = (ne_lon - sw_lon) / panels_in_width

    # Calculate and store panel centers along with their latitudes and longitudes
    panel_centers = []
    for i in range(panels_in_height):
        for j in range(panels_in_width):
            center_lat = sw_lat + i * lat_step + lat_step / 2
            center_lon = sw_lon + j * lon_step + lon_step / 2
            panel_centers.append((center_lat, center_lon))
            #print(f"Panel Center {len(panel_centers)} - Latitude: {center_lat}, Longitude: {center_lon}")

    return panel_centers

# Extract main bounding box
main_box_data = data['boundingBox']
main_box = [
    main_box_data['sw']['latitude'],
    main_box_data['sw']['longitude'],
    main_box_data['ne']['latitude'],
    main_box_data['ne']['longitude']
]


# Calculate the number of panels that can fit inside the main bounding box
total_panels_in_main_box = panels_in_bounding_box(main_box, panel_width, panel_height)

# Calculate panel centers and print their latitudes and longitudes
panel_centers = calculate_panel_centers(main_box, panel_width, panel_height)

# Plotting code
plt.figure(figsize=(10, 10))
plt.plot([main_box[1], main_box[3], main_box[3], main_box[1], main_box[1]], 
         [main_box[0], main_box[0], main_box[2], main_box[2], main_box[0]], 'k-', label='Main Bounding Box')

# Plot panel centers
for center in panel_centers:
    plt.scatter(center[1], center[0], c='b', marker='.', s=10)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Panel Centers Inside Main Bounding Box')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()
# Print the result
print("Total number of panels that can fit inside the main bounding box:", total_panels_in_main_box)

# Function to calculate distances between neighboring panel centers
def calculate_distances(panel_centers):
    num_centers = len(panel_centers)
    min_distances = {}

    for i in range(num_centers):
        for j in range(i + 1, num_centers):
            lat1, lon1 = panel_centers[i]
            lat2, lon2 = panel_centers[j]
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            min_distances[(i, j)] = distance

    return min_distances


# Calculate minimum distances between neighboring panel centers in inches
distances = calculate_distances(panel_centers)

# Find the minimum distance
min_distance = min(distances.values())

# Print the minimum distance in inches
print(f"Minimum distance between neighboring panels: {min_distance} inches")

In [ ]:
 # Extract 'roofSegmentStats' from the 'solarPotential_data'
roof_segment_stats = solarPotential_data.get('roofSegmentStats', [])

# Initialize an empty list to hold the samples
samples = []

# Initialize a dictionary to store the bounding boxes for each sample
sample_bounding_boxes = {}

# Loop through each sample and extract the bounding box information
for i, segment in enumerate(roof_segment_stats):
    bounding_box = segment.get('boundingBox', {})
    sw_lat = bounding_box.get('sw', {}).get('latitude', None)
    sw_lon = bounding_box.get('sw', {}).get('longitude', None)
    ne_lat = bounding_box.get('ne', {}).get('latitude', None)
    ne_lon = bounding_box.get('ne', {}).get('longitude', None)

    if sw_lat is not None and sw_lon is not None and ne_lat is not None and ne_lon is not None:
        sample_name = f'Sample_{i+1}'
        sample_bounding_boxes[sample_name] = (sw_lat, sw_lon, ne_lat, ne_lon)
        #print(f"Sample {i+1} - SW: ({sw_lat}, {sw_lon}), NE: ({ne_lat}, {ne_lon})")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

# Declare panel_df as global
global panel_df

def assign_panel_numbers_and_colors(panel_centers, sample_bounding_boxes, sample_heights):
    global panel_df  # Indicate that we are using the global variable
    panel_info = []
    panel_numbers = {}
    panel_colors = {}
    
    for i, panel_center in enumerate(panel_centers):
        #print(f"Checking panel at {panel_center}")
        lat, lon = panel_center
        assigned_sample = None
        highest_sample_height = -1

        for j, (sample_name, bounding_box) in enumerate(sample_bounding_boxes.items()):
            #=print(f"Panel at {lat}, {lon} is being checked against {sample_name}")
            sw_lat, sw_lon, ne_lat, ne_lon = bounding_box
            #print(f"sw_lat: {sw_lat}, sw_lon: {sw_lon}, ne_lat: {ne_lat}, ne_lon: {ne_lon}, lat: {lat}, lon: {lon}")
            if sw_lat <= lat <= ne_lat and sw_lon <= lon <= ne_lon:
                print(f"Panel at {lat}, {lon} is within bounding box {bounding_box}")
                if sample_heights[j] > highest_sample_height:
                    print(f"Panel at {lat}, {lon} is assigned to {sample_name}")
                    assigned_sample = sample_name
                    if sample_heights[j] > highest_sample_height:
                        print(f"the highest changed from {highest_sample_height} to {sample_heights[j]}")
                    highest_sample_height = sample_heights[j]

        if assigned_sample:
            #print( f"Panel at {lat}, {lon} is assigned to {assigned_sample}")
            panel_number = len(panel_numbers) + 1
            panel_numbers[panel_center] = panel_number
            panel_colors[panel_center] = cm.tab20(i)
            panel_info.append([panel_number, assigned_sample, panel_center, highest_sample_height])
            
    panel_df = pd.DataFrame(panel_info, columns=['Panel Number', 'Sample Assigned', 'Panel Center', 'Panel Height'])
    
    # Filter out unassigned panels
    panel_df = panel_df[panel_df['Sample Assigned'].notna()]
    
    return panel_numbers, panel_colors

# Call the function
panel_numbers, panel_colors = assign_panel_numbers_and_colors(panel_centers, sample_bounding_boxes, sample_heights)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

# Assign panel numbers and colors
def assign_panel_numbers_and_colors(panel_centers, sample_bounding_boxes, sample_heights):
    panel_numbers = {}
    panel_colors = {}
    
    for i, panel_center in enumerate(panel_centers):
        lat, lon = panel_center
        assigned_sample = None
        highest_sample_height = -1
        color_index = -1

        for j, (sample_name, bounding_box) in enumerate(sample_bounding_boxes.items()):
            sw_lat, sw_lon, ne_lat, ne_lon = bounding_box
            if sw_lat <= lat <= ne_lat and sw_lon <= lon <= ne_lon:
                if sample_heights[j] > highest_sample_height:
                    assigned_sample = sample_name
                    highest_sample_height = sample_heights[j]
                    color_index = j

        if assigned_sample:
            panel_number = len(panel_numbers) + 1
            panel_numbers[panel_center] = panel_number
            color = cm.tab20(color_index)  
            panel_colors[panel_center] = color
    
    return panel_numbers, panel_colors

# Call the function
panel_numbers, panel_colors = assign_panel_numbers_and_colors(panel_centers, sample_bounding_boxes, sample_heights)

# Plotting
for panel_center, panel_number in panel_numbers.items():
    lat, lon = panel_center
    color = panel_colors.get(panel_center, 'blue')
    plt.scatter(lon, lat, c=[color], s=50)
    plt.annotate(f'Panel {panel_number}', (lon, lat), fontsize=8, color='black', ha='center', va='center')

# Plot sample bounding boxes with unique colors
for i, (sample_name, bounding_box) in enumerate(sample_bounding_boxes.items()):
    sw_lat, sw_lon, ne_lat, ne_lon = bounding_box
    color = cm.tab20(i)
    height = sample_heights[i]  # Extract the height for this sample
    plt.plot([sw_lon, ne_lon, ne_lon, sw_lon, sw_lon], [sw_lat, sw_lat, ne_lat, ne_lat, sw_lat], color=color, label=f'{sample_name} (Height: {height} meters)')

plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Initialize a dictionary to store sunshine quantiles for each sample
sunshine_quantiles_dict = {}

# Loop through each sample and extract sunshine quantile information
for i, segment in enumerate(roofSegmentStats):
    stats = segment.get('stats', {})
    sunshine_quantiles = stats.get('sunshineQuantiles', [])
    sample_name = f'Sample_{i+1}'
    sunshine_quantiles_dict[sample_name] = sunshine_quantiles
#print(sunshine_quantiles_dict)

# Create a new column for average sunshine quantiles in panel_df
panel_df['Average Sunshine Quantile'] = 0

# Loop through each row in panel_df to assign the average sunshine quantile
for index, row in panel_df.iterrows():
    sample_name = row['Sample Assigned']
    sunshine_quantiles = sunshine_quantiles_dict.get(sample_name, [])
    
    if sunshine_quantiles:
        average_sunshine_quantile = sum(sunshine_quantiles) / len(sunshine_quantiles)
        panel_df.loc[index, 'Average Sunshine Quantile'] = average_sunshine_quantile

panel_df = panel_df.sort_values(by=['Average Sunshine Quantile'], ascending=False)
panel_df.head(10)


In [ ]:
# sum of average sunshine quantiles
sum(panel_df['Average Sunshine Quantile'])
print (f"sum of average sunshine quantiles: {sum(panel_df['Average Sunshine Quantile'])}")
print (f"total number of panels: {len(panel_df)}")